In [2]:
import requests
import json
import base64

AI_API_BASE = "https://api.thucchien.ai"
AI_API_KEY = "sk-bN54VOyPuJFk2S4YhAHH8Q"

In [9]:
import json

with open("prompts.json", "r", encoding="utf-8") as f:
    data = json.load(f)

GENERATION_PROMPT = data["generation_prompt"]
IDEA_PROMPT = data["idea_prompt"]
CODE_SPECIALIST_PROMPT = data["code_specialist"]

In [3]:
def get_key_info():
    url = "https://api.thucchien.ai/key/info"

    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {AI_API_KEY}"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        key_info = response.json()
        info = key_info['info']
        #   print("Key Alias:", info['key_alias'])
        print("Spent: $" + str(info['spend']) + " / $" + str(info['max_budget'] if 'max_budget' in info else 'Unlimited'))
        #   print("Max Budget:", info.get('max_budget', 'Unlimited'))
        #   print("Models:", info['models'])
        #   print("Created:", info['created_at'])
    else:
        print("Error:", response.status_code)
        print(response.text)

In [14]:
get_key_info()

Spent: $5.74427505 / $50.0


In [5]:

import base64
import os
import wave
import struct

def generate_image(prompt, n_images=1, out_dir="generated_images", aspect_ratio="1:1"):
  # --- Gọi API để tạo hình ảnh ---
  url = f"{AI_API_BASE}/images/generations"
  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {AI_API_KEY}"
  }
  data = {
    "model": "imagen-4",
    "prompt": prompt,
    "n": n_images, # Yêu cầu 2 ảnh
    "aspect_ratio": aspect_ratio
  }

  try:
    response = requests.post(url, headers=headers, data=json.dumps(data))
    response.raise_for_status()

    get_key_info()

    result = response.json()
    
    # --- Xử lý và lưu từng ảnh ---
    for i, image_obj in enumerate(result['data']):
        b64_data = image_obj['b64_json']
        image_data = base64.b64decode(b64_data)
        
        save_path = f"{out_dir}_{i+1}.png"
        with open(save_path, 'wb') as f:
            f.write(image_data)
        print(f"Image saved to {save_path}")

  except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
    print(f"Response body: {response.text if 'response' in locals() else 'No response'}")

In [6]:
# Add masked image
def edit_image(prompt, image_path, aspect_ratio="1:1", out_path="edited_image.png"):
    """
    Edit an existing image using ThucChien.ai Gemini API (v1beta).
    
    Args:
        prompt (str): Edit instruction (e.g., "Remove all text from the image").
        image_path (str): Path to the original image.
        api_key (str): Your ThucChien.ai API key.
        aspect_ratio (str): Target aspect ratio, e.g. "1:1", "9:16".
        out_path (str): Path to save the edited image.
    """

    # --- Read and encode the original image ---
    with open(image_path, "rb") as f:
        image_bytes = f.read()
    image_b64 = base64.b64encode(image_bytes).decode("utf-8")

    # --- API request setup ---
    url = "https://api.thucchien.ai/gemini/v1beta/models/gemini-2.5-flash-image-preview:generateContent"
    headers = {
        "x-goog-api-key": AI_API_KEY,
        "Content-Type": "application/json"
    }

    # --- Payload structure for image editing ---
    payload = {
        "contents": [
            {
                "parts": [
                    {"text": prompt},
                    {
                        "inline_data": {
                            "mime_type": "image/png",
                            "data": image_b64
                        }
                    }
                ]
            }
        ],
        "generationConfig": {
            "imageConfig": {
                "aspectRatio": aspect_ratio
            }
        }
    }

    # --- Send request ---
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    get_key_info()
    try:
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        print(f"Response: {response.text}")
        return None

    result = response.json()

    # --- Extract base64 image from response ---
    try:
        img_b64 = result["candidates"][0]["content"]["parts"][1]["inlineData"]["data"]
        # img_b64 = result["candidates"][0]["content"]["parts"]
        # return img_b64
    except KeyError:
        print(img_b64)
        print("⚠️ Could not find image data in response.")
        print(json.dumps(result, indent=2))
        return None

    img_bytes = base64.b64decode(img_b64)

    # --- Save edited image ---
    os.makedirs(os.path.dirname(out_path) or ".", exist_ok=True)
    with open(out_path, "wb") as f:
        f.write(img_bytes)

    print(f"✅ Edited image saved to {out_path}")
    return out_path

In [10]:
edit_image(prompt="Help me generate image about a website UI based on the following based website design. The website is an information website about a famous attraction", image_path="raw.png", aspect_ratio="16:9", out_path="edited_website.png")

Spent: $0.263287175 / $50.0
✅ Edited image saved to edited_website.png


'edited_website.png'

In [11]:
edit_image(prompt="Help me generate image about a website UI based on the following based website design. The website is an information website about a famous attraction, which is Bản Yên Hoà, Xã Mỹ Lý, Tỉnh Nghệ An. This place has the following characteristic: Tháp cổ linh thiêng, Món ăn hấp dẫn, Múa xòe dân tộc Thái, Làng nghề dệt thổ cẩm, Món ăn đặc sản người Thái: cơm lam, cá nướng, gà nướng, xôi nếp cẩm, Du thuyền trên sông Nậm, Cầu treo yên hòa: cầu treo dân sinh dài nhất huyện kỳ sơn, Tháp cổ Yên Hòa: Tháp Xốp lợt", image_path="raw.png", aspect_ratio="16:9", out_path="edited_website.png")

Spent: $0.3971203 / $50.0
✅ Edited image saved to edited_website.png


'edited_website.png'

In [21]:
get_key_info()

Spent: $17.94245286250001 / $50.0
